In [1]:
from os.path import join
import pandas as pd
import torch
from tqdm.auto import tqdm
import pickle
from transformers import XLMTokenizer, RobertaModel

In [2]:
from transformers import XLMTokenizer, RobertaModel

tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

In [3]:
json_df_path = join("..", "data", "slowosiec_all_data.json.gz")

df = pd.read_json(json_df_path)

save_path = join("..", "data", "slowosiec_all_data_word_level_embeddingss.pickle")

In [18]:
DEVICE = "cuda"

# tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
tokenizer = XLMTokenizer.from_pretrained("./models/tokenizer/")

# model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")
model = RobertaModel.from_pretrained("./models/bert/")

model = model.to(DEVICE)

In [21]:
# model.save_pretrained("./models/bert/")
# tokenizer.save_pretrained("./models/tokenizer/")

In [20]:
# tokenizer.save_pretrained("./models/tokenizer/")

In [12]:
from typing import Tuple, List, Dict
import numpy as np

def calculate_embeddings(
    texts: List[str],
    labels: List[str],
    emote_to_text: Dict[str, str],
) -> Tuple[List[str], List[np.ndarray], List[np.ndarray], List[str]]:

    processed_texts = []
    sentence_embeddings_list = []
    seq_embeddings_list = []
    processed_labels = []
    
    for text, label in tqdm(zip(texts, labels), desc='Encoding texts'):
        try:
            text = _replace_emotes_with_text(text, emote_to_text)
            tokenized = tokenizer.encode(text, return_tensors='pt').to(DEVICE)

            with torch.no_grad():
                outputs = model(tokenized)
                
                seq_embedding = outputs[0].squeeze(dim=0).cpu()
                sentence_embedding = outputs[1].squeeze(dim=0).cpu()
            
            processed_texts.append(text)
            sentence_embeddings_list.append(sentence_embedding)
            seq_embeddings_list.append(seq_embedding)
            processed_labels.append(label)

        except Exception:
            pass

    return processed_texts, sentence_embeddings_list, seq_embeddings_list, processed_labels

In [ ]:
def get_texts_embeddings_and_labels(texts):
    texts = []
    
    sentence_embeddings = []
    seq_embeddings = []
    
    for emotion, text in tqdm():
        encoded_input = tokenizer.encode(text, return_tensors='pt').to("cuda")
        with torch.no_grad():
            outputs = model(encoded_input)
            sentence_embedding = outputs[1].squeeze(dim=0).cpu()

        texts.append(text)
        sentence_embeddings.append(sentence_embedding)
        emotions.append(emotion)
    
    

In [ ]:
def get_texts_embeddings_and_labels(texts):
    texts = []
    sentence_embeddings = []
    seq_embeddings = []
    
    emotions = []

    for emotion, text in tqdm(zip(df["('emotions',)"], df["('text',)"])):
        encoded_input = tokenizer.encode(text, return_tensors='pt').to("cuda")
        with torch.no_grad():
            outputs = model(encoded_input)
            sentence_embedding = outputs[1].squeeze(dim=0).cpu()

        texts.append(text)
        sentence_embeddings.append(sentence_embedding)
        emotions.append(emotion)
    
    

In [19]:


texts = []
sentence_embeddings = []
emotions = []

for emotion, text in tqdm(zip(df["('emotions',)"], df["('text',)"])):
    encoded_input = tokenizer.encode(text, return_tensors='pt').to("cuda")
    with torch.no_grad():
        outputs = model(encoded_input)
        sentence_embedding = outputs[1].squeeze(dim=0).cpu()
    
    texts.append(text)
    sentence_embeddings.append(sentence_embedding)
    emotions.append(emotion)

KeyboardInterrupt: 